<a href="https://colab.research.google.com/github/Christopher-Ramirez-S/An-lisis-de-datos/blob/Adrian/lab4-adr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
import requests
import json
import concurrent.futures
import threading
import time
import logging
import os
from tqdm import tqdm
import datetime
import pandas as pd
import csv
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = pd.read_csv('/content/TMDB_movie_dataset_v11.csv')
data = pd.DataFrame(df)


In [10]:
data['release_date']=pd.to_datetime(data['release_date'],errors='coerce')
data_con_fechas = data.dropna(subset=['release_date'])
pelicula_vieja = data_con_fechas[data_con_fechas['release_date'] == data_con_fechas['release_date'].min()]
print("Pelicula mas vieja")
print(pelicula_vieja)
print("fecha de lanzamiento")
print(data['release_date'])

Pelicula mas vieja
             id                                title  vote_average  \
603006  1294302  בראול סטארס בחיים האמיתיים-אלון קאט           0.0   

        vote_count    status release_date    revenue  runtime  adult  \
603006           0  Released   1800-01-01  100000000        0  False   

       backdrop_path  ...                       original_title overview  \
603006           NaN  ...  בראול סטארס בחיים האמיתיים-אלון קאט      NaN   

       popularity                       poster_path tagline  \
603006        0.6  /bCKTrWbglTHpTgl3crAhHZg7TgK.jpg     NaN   

                                       genres  production_companies  \
603006  Horror, Drama, Thriller, Mystery, War            Salty Fish   

       production_countries spoken_languages keywords  
603006                  NaN              NaN      NaN  

[1 rows x 24 columns]
fecha de lanzamiento
0         2010-07-15
1         2014-11-05
2         2008-07-16
3         2009-12-15
4         2012-04-25
             

In [11]:
start_date = pd.to_datetime('1910-01-01')
end_date = pd.to_datetime('2024-07-31')
filtered_data = data[(data['release_date'] >= start_date) & (data['release_date'] <= end_date)]
filtered_data

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085995,756658,Unarchive,0.000,0,Released,2020-04-03,0,13,False,NaN,...,Desarquivar,"""Unarchive"" juxtaposes the filmmaker's father'...",0.600,/eCyzs7XJqjiXdVosJrCXExc6K5W.jpg,NaN,Documentary,NaN,"Canada, Chile",NaN,NaN
1085996,756660,We are the world,0.000,0,Released,1988-11-01,0,0,False,NaN,...,我们是世界,"In 1940, the desert areas of northwest China s...",0.600,/t8SLrkJNaehU0GLJCgXnvKxPU7s.jpg,NaN,"Drama, History","Xi'an Film Studio, Ningxia Film Studio",China,Mandarin,NaN
1085997,756661,Alien Worlds: Giants and Hybrids,0.000,0,Released,2020-10-20,0,60,False,/gqvkcgqKFVdXRrK8OJKqWjMdZIk.jpg,...,Alien Worlds: Giants and Hybrids,"Explore evidence of colossal, biblical Giants ...",1.733,/hcpiY0Gwhxx8cHHW4iXLo7VozPu.jpg,NaN,Documentary,NaN,United States of America,English,NaN
1085998,756662,"""Sok húron pendülnek""",0.000,0,Released,1982-04-01,0,61,False,/dsh08fQcM8iRofFy5KJChBgxJjq.jpg,...,"""Sok húron pendülnek""",Hungarian documentary from 1982.,0.600,/eEdOgdt7lbOd032GqlXtC3daA3M.jpg,NaN,Documentary,NaN,NaN,Hungarian,NaN


In [ ]:
##Adrian query
#Sacar las 10 peliculas con mayor votacion en todos los años

MayorVot=filtered_data.sort_values(by='vote_count',ascending=False)
MayorVot.head(10)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
5,293660,Deadpool,7.606,28894,Released,2016-02-09,783100000,108,False,/en971MEXui9diirXlogOrPKmsEn.jpg,...,Deadpool,The origin story of former Special Forces oper...,72.735,/zq8Cl3PNIDGU3iWNRoc5nEZ6pCe.jpg,Witness the beginning of a happy ending.,"Action, Adventure, Comedy","20th Century Fox, The Donners' Company, Genre ...",United States of America,English,"superhero, anti hero, mercenary, based on comi..."
6,299536,Avengers: Infinity War,8.255,27713,Released,2018-04-25,2052415039,149,False,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,...,Avengers: Infinity War,As the Avengers and their allies have continue...,154.340,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,An entire universe. Once and for all.,"Adventure, Action, Science Fiction",Marvel Studios,United States of America,"English, Xhosa","sacrifice, magic, superhero, based on comic, s..."
7,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,False,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,Fight Club,A ticking-time-bomb insomniac and a slippery s...,69.498,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Mischief. Mayhem. Soap.,Drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...",United States of America,English,"dual identity, rage and hate, based on novel o..."
8,118340,Guardians of the Galaxy,7.906,26638,Released,2014-07-30,772776600,121,False,/uLtVbjvS1O7gXL8lUOwsFOH4man.jpg,...,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",33.255,/r7vmZjiyZw9rpJMQJdXpjgiCOk9.jpg,All heroes start somewhere.,"Action, Science Fiction, Adventure",Marvel Studios,United States of America,English,"spacec

In [ ]:
#Adrian query
#Suma la cantidad de votos que a recibido todas las peliculas de una productora, las 10 primeras de mayor a menor

VotosPorProd = filtered_data.groupby('production_companies')['vote_count'].sum()
VotosPorProd.sort_values(ascending=False).head(10)

,vote_count
production_companies,
Marvel Studios,433824
"Pixar, Walt Disney Pictures",169137
Paramount,115439
Walt Disney Productions,112438
Pixar,110702
Lucasfilm Ltd.,100019
"Warner Bros. Pictures, Heyday Films",87288
"Walt Disney Pictures, Walt Disney Feature Animation",79750
Universal Pictures,77460


In [ ]:
#Adrian query
# Filtra todas las peliculas que esten en idioma Ingles
PeliIngles = filtered_data[filtered_data['spoken_languages'].str.contains('En', na=False)]
PeliIngles


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085979,756669,Sweets And Feet,0.000,0,Released,2014-06-20,0,0,True,NaN,...,Sweets And Feet,NaN,0.600,/ieUXSBLb6IKl1ep8LIfBXAINZVU.jpg,NaN,NaN,GayLife Network,United States of America,English,gay pornography
1085983,284046,9 Days in Summer,0.000,0,Released,1967-01-01,0,48,False,NaN,...,9 Days in Summer,A promotional film for the Ford Company detail...,0.833,/vPrpapFCFUgAEDbFBPu6UHEctuV.jpg,NaN,Documentary,"Ford Film Unit, Philip Bond and Partners",United Kingdom,English,"engineering, ford, cosworth, formula one (f1)"
1085990,756648,Bunker,0.000,0,Released,2007-09-01,0,90,True,NaN,...,Bunker,SLEAZE: tastelessness by virtue of being cheap...,0.600,/lTN8GORAFJQ2wldOWcc3iXhbBAv.jpg,NaN,NaN,Mustang Studios,United States of America,English,gay pornography
1085993,756656,The Story of Lassie,0.000,0,Released,1994-08-14,0,60,False,NaN,...,The Story of Lassie,"Documentary about canine superstar Lassie, com...",1.440,/7jB1zMCaMB4EgMY53jHSzEK4Lhz.jpg,NaN,"Documentary, Family",Wombat Productions,United States of America,English,NaN


Lab-4

Estudiantes:
Adrian Rodriguez Rodriguez,
Christopher Ramirez Suarez,
Aaron Navarro Balmaceda,
Rafael Garcia Delgado,
eldon caldwell salazar,


In [12]:
import pandas as pd

estudiantes = [('Ana', 23, 'San Jose', 'A'),
            ('Esteban', 22, 'Heredia', 'B'),
            ('Juan', 22, 'Cartago', 'A'),
            ('Karla', 22, 'Limón', 'A'),
            ('Ernesto', 21, 'Heredia', 'B')
            ]

df = pd.DataFrame(estudiantes, columns =['Nombre', 'Edad', 'Ciudad', 'Seccion'])
print(df)

    Nombre  Edad    Ciudad Seccion
0      Ana    23  San Jose       A
1  Esteban    22   Heredia       B
2     Juan    22   Cartago       A
3    Karla    22     Limón       A
4  Ernesto    21   Heredia       B


Funcion a retornar conjunto de datos


In [2]:
def mostrar_datos(df, columna):
  print(df[columna])

mostrar_datos(df, 'Edad')

0    23
1    22
2    22
3    22
4    21
Name: Edad, dtype: int64


Funcion que devulve la ubicacion de los elementos de la bus queda

In [3]:
def getIndexes(dfObj, value):

    listOfPos = []
    result = dfObj.isin([value])
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)

    for col in columnNames:
        rows = list(result[col][result[col] == True].index)

        for row in rows:
            listOfPos.append((row, col))

    return listOfPos

In [16]:

listOfPositions = getIndexes(filtered_data, '1980-01-01')

print('Index positions of 22 in Dataframe : ')

for i in range(len(listOfPositions)):
    print( listOfPositions[i])

Index positions of 22 in Dataframe : 
(23207, 'release_date')
(26195, 'release_date')
(27989, 'release_date')
(31502, 'release_date')
(33225, 'release_date')
(37530, 'release_date')
(37908, 'release_date')
(38529, 'release_date')
(38719, 'release_date')
(39448, 'release_date')
(47291, 'release_date')
(47557, 'release_date')
(47947, 'release_date')
(51037, 'release_date')
(56984, 'release_date')
(60390, 'release_date')
(60717, 'release_date')
(61551, 'release_date')
(62068, 'release_date')
(62795, 'release_date')
(65012, 'release_date')
(65391, 'release_date')
(67822, 'release_date')
(72176, 'release_date')
(73985, 'release_date')
(77714, 'release_date')
(77759, 'release_date')
(79644, 'release_date')
(81606, 'release_date')
(83566, 'release_date')
(83970, 'release_date')
(84597, 'release_date')
(88460, 'release_date')
(88562, 'release_date')
(90916, 'release_date')
(91308, 'release_date')
(91589, 'release_date')
(92884, 'release_date')
(93602, 'release_date')
(96317, 'release_date')
(9

In [17]:
listOfPositions = getIndexes(filtered_data, 'Marvel Studios')

print('Index positions of 22 in Dataframe : ')

for i in range(len(listOfPositions)):
    print( listOfPositions[i])

Index positions of 22 in Dataframe : 
(4, 'production_companies')
(6, 'production_companies')
(8, 'production_companies')
(12, 'production_companies')
(15, 'production_companies')
(23, 'production_companies')
(24, 'production_companies')
(26, 'production_companies')
(27, 'production_companies')
(28, 'production_companies')
(34, 'production_companies')
(36, 'production_companies')
(39, 'production_companies')
(42, 'production_companies')
(52, 'production_companies')
(67, 'production_companies')
(82, 'production_companies')
(117, 'production_companies')
(169, 'production_companies')
(320, 'production_companies')
(373, 'production_companies')
(451, 'production_companies')
(737, 'production_companies')
(5126, 'production_companies')
(6523, 'production_companies')
(6640, 'production_companies')
(6777, 'production_companies')
(7355, 'production_companies')
(8881, 'production_companies')
(9249, 'production_companies')
(10556, 'production_companies')
(10707, 'production_companies')
(10907, 'pr